In [ ]:
from pyscbwrapper import SCB

In [ ]:

scb = SCB('en')

In [ ]:
scb.info()

In [ ]:
scb.go_down('TK')

In [ ]:
scb.info()

In [ ]:
scb.go_down('TK1001')
scb.info()

In [ ]:
scb.go_down('TK1001A')
scb.info()

In [ ]:
scb.go_down('PersBilarDrivMedel')
scb.info()

In [ ]:
scb.get_variables()

In [ ]:
import requests
import json

In [ ]:
session = requests.Session()

In [ ]:
query = {
  "query": [
    {
      "code": "Drivmedel",
      "selection": {
        "filter": "item",
        "values": [
          "100",
          "110"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}
url = "https://api.scb.se/OV0104/v1/doris/en/ssd/START/TK/TK1001/TK1001A/PersBilarDrivMedel"

response = session.post(url, json=query)

In [ ]:
print(response)

In [ ]:
response_json = json.loads(response.content.decode('utf-8-sig'))

In [ ]:
response_json

In [ ]:
import pandas as pd

In [ ]:
# Create an empty list to store our rows
data_rows = []

# Loop through each item in the 'data' list
for item in response_json['data']:
    # The 'key' is a list of [Region, FuelType, YearMonth]
    region = item['key'][0]
    fuel_type = item['key'][1]
    year_month = item['key'][2]
    
    # The 'values' is a list with the number of registrations
    number_of_registrations = int(item['values'][0]) # Convert to integer
    
    # Append the extracted data as a dictionary to our list
    data_rows.append({
        'YearMonth': year_month,
        'Region': region,
        'FuelType': fuel_type,
        'NumberOfRegistration': number_of_registrations
    })

# Create the pandas DataFrame from our list of dictionaries
df = pd.DataFrame(data_rows)

In [ ]:
df.head(5)  # Preview first five entries

In [ ]:
df.shape

In [ ]:
df['FuelType'].value_counts()

In [ ]:
df['Region'].value_counts()

In [23]:
import requests
import json

In [24]:
session = requests.Session()

In [ ]:
query = {
  "query": [
    {
      "code": "Drivmedel",
      "selection": {
        "filter": "item",
        "values": [
          "110"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
url = "https://api.scb.se/OV0104/v1/doris/en/ssd/START/TK/TK1001/TK1001A/PersBilarDrivMedel"

response = session.post(url, json=query)

In [26]:
print(response)

<Response [200]>


In [27]:
data = response.json()

In [28]:
import pandas as pd

In [29]:

data = response.json()

# Step 2: Extract dimension labels
regions = data['dimension']['Region']['category']['label']
fuels = data['dimension']['Drivmedel']['category']['label']
months = data['dimension']['Tid']['category']['label']

# You used only one fuel, so get the label directly
fuel_label = list(fuels.values())[0]

# Step 3: Flatten into DataFrame
region_codes = list(regions.keys())
month_codes = list(months.keys())
values = data['value']

records = []
index = 0

for region_code in region_codes:
    for month_code in month_codes:
        records.append({
            "Region": regions[region_code],
            "FuelType": fuel_label,
            "YearMonth": months[month_code],
            "NumberOfRegistration": values[index]
        })
        index += 1

df = pd.DataFrame(records)

# Step 4: Done!
print(df.head())

   Region FuelType YearMonth  NumberOfRegistration
0  Sweden   petrol   2006M01                 13617
1  Sweden   petrol   2006M02                 14687
2  Sweden   petrol   2006M03                 19609
3  Sweden   petrol   2006M04                 21439
4  Sweden   petrol   2006M05                 24117


In [30]:
df["FuelType"].value_counts()

FuelType
petrol    74025
Name: count, dtype: int64

In [ ]:
df.shape